# **Perform inference**

## **1. Description**

Here, I use the model to perform query questions.

Each question will assess how different compartments of the predation process influence prey capture.

- **Question 1 :** How does expertise influence foraging decisions and prey capture?

- **Question 2 :** How does past experience and memory influence foraging decisions and prey capture?

- **Question 3 :** How does the current interaction influence foraging decision and prey capture?

I will perform approximate inference using sampling to generate probability distributions for my queries.

Each query will return a conditional probability table.

## **2. Prepare the session**

Load the libraries

In [1]:
# To manipulate data
import pandas as pd

# To perform inference
from pgmpy.inference import ApproxInference

# To import the data
from joblib import load
import os

Import the model object.

In [2]:
# Define the parent directory
parent_dir = os.path.abspath(
    os.path.join(os.getcwd(), os.pardir)
)

# Define the subfolder directory
output_folder = os.path.join(
    parent_dir, "outputs"
)

# Name of the model output file
file_name = "BayesNet-fit.joblib"

model = load(
    os.path.join(
        output_folder,
        file_name
    )
)

## **3. Question 1 - Expertise**

I start by creating an inference object called "infer".

In [3]:
infer = ApproxInference(model)

### **3.1 Foraging mode**

#### **3.1.1 Query 1**

Here, we ask the question whether the predator's foraging mode changes with experience given longer time lags between sessions.

In [8]:
q1a = infer.query(
    variables = ["pred_speed_bin", "xp_level"],
    evidence = {"time_lag_session_log_bin": "Q5"},
    joint = True,
    n_samples = 5000,
    seed = 123,
    show_progress = False
)

In [9]:
print(q1a)

+-------------------------+--------------------+--------------------------------+
| pred_speed_bin          | xp_level           |   phi(pred_speed_bin,xp_level) |
+=========================+====================+================================+
| pred_speed_bin(Average) | xp_level(interm)   |                         0.3362 |
+-------------------------+--------------------+--------------------------------+
| pred_speed_bin(Average) | xp_level(advanced) |                         0.2192 |
+-------------------------+--------------------+--------------------------------+
| pred_speed_bin(Average) | xp_level(novice)   |                         0.1654 |
+-------------------------+--------------------+--------------------------------+
| pred_speed_bin(Slower)  | xp_level(interm)   |                         0.0754 |
+-------------------------+--------------------+--------------------------------+
| pred_speed_bin(Slower)  | xp_level(advanced) |                         0.0624 |
+---------------

**Result**. Chances are higher that a predator will a dopt an intercepting hunting mode at any experience level when the time lag is high.

Given this result, we now perform the same query, but when the time lag is very short.

In [10]:
q1b = infer.query(
    variables = ["pred_speed_bin", "xp_level"],
    evidence = {"time_lag_session_log_bin": "Q1"},
    joint = True,
    n_samples = 5000,
    seed = 123,
    show_progress = False
)

In [11]:
print(q1b)

+-------------------------+--------------------+--------------------------------+
| pred_speed_bin          | xp_level           |   phi(pred_speed_bin,xp_level) |
+=========================+====================+================================+
| pred_speed_bin(Average) | xp_level(advanced) |                         0.2066 |
+-------------------------+--------------------+--------------------------------+
| pred_speed_bin(Average) | xp_level(interm)   |                         0.3314 |
+-------------------------+--------------------+--------------------------------+
| pred_speed_bin(Average) | xp_level(novice)   |                         0.1588 |
+-------------------------+--------------------+--------------------------------+
| pred_speed_bin(Slower)  | xp_level(advanced) |                         0.0698 |
+-------------------------+--------------------+--------------------------------+
| pred_speed_bin(Slower)  | xp_level(interm)   |                         0.0954 |
+---------------

Overall, it seems that at any experience level and regardless of the time lag between sessions (here, low time lag), predators are more likely to use an intercepting strategy. Chances are also higher that they use such strategy when they are intermediate hunters.

### **3.2 Chase rate**

#### **3.2.1 Query 2.**

We now perform a similar query, but looking at the predator's chase rate. The hypothesis is that higher experience should lead to more chases. Alternatively, lower chase rates with experience may indicate higher performance because of optimal strategies. We will adress this second prediction later in query 3.

In [12]:
q2a = infer.query(
    variables = ["chases_persec_bin", "xp_level"],
    evidence = {"time_lag_session_log_bin": "Q5"},
    joint = True,
    n_samples = 5000,
    seed = 123,
    show_progress = False
)

In [13]:
print(q2a)

+----------------------------+--------------------+-----------------------------------+
| chases_persec_bin          | xp_level           |   phi(chases_persec_bin,xp_level) |
+============================+====================+===================================+
| chases_persec_bin(Average) | xp_level(interm)   |                            0.2468 |
+----------------------------+--------------------+-----------------------------------+
| chases_persec_bin(Average) | xp_level(advanced) |                            0.1696 |
+----------------------------+--------------------+-----------------------------------+
| chases_persec_bin(Average) | xp_level(novice)   |                            0.1124 |
+----------------------------+--------------------+-----------------------------------+
| chases_persec_bin(High)    | xp_level(interm)   |                            0.0532 |
+----------------------------+--------------------+-----------------------------------+
| chases_persec_bin(High)    | x

Similar to the foraging mode, chances are higher that a predator will have average chase rates when they are intermediate hunters, but also regardless of their experience level. Interestingly, there are also higher chances that chase rates will be low when predators are intermediate and advanced, suggesting that some predators increase their performance.

### **3.3 Prey capture**

#### **3.3.1 Query 3**

We now perform queries for the joint probability of prey capture and experience given time lags

In [19]:
q3a = infer.query(
    variables = ["prey_captured", "xp_level"],
    evidence = {"time_lag_session_log_bin": "Q5"},
    joint = True,
    n_samples = 5000,
    seed = 123,
    show_progress = False
)

In [20]:
print(q3a)

+------------------+--------------------+-------------------------------+
| prey_captured    | xp_level           |   phi(prey_captured,xp_level) |
+==================+====================+===============================+
| prey_captured(1) | xp_level(interm)   |                        0.3568 |
+------------------+--------------------+-------------------------------+
| prey_captured(1) | xp_level(advanced) |                        0.2424 |
+------------------+--------------------+-------------------------------+
| prey_captured(1) | xp_level(novice)   |                        0.1804 |
+------------------+--------------------+-------------------------------+
| prey_captured(0) | xp_level(interm)   |                        0.1004 |
+------------------+--------------------+-------------------------------+
| prey_captured(0) | xp_level(advanced) |                        0.0718 |
+------------------+--------------------+-------------------------------+
| prey_captured(0) | xp_level(novice) 

Chances of capturing prey remain relatively high even when the time lag between sessions is long. However, chances of capturing prey are lower when predators are novice.

We now perform the same query but for short time lags.

In [21]:
q3b = infer.query(
    variables = ["prey_captured", "xp_level"],
    evidence = {"time_lag_session_log_bin": "Q1"},
    joint = True,
    n_samples = 5000,
    seed = 123,
    show_progress = False
)

In [22]:
print(q3b)

+------------------+--------------------+-------------------------------+
| prey_captured    | xp_level           |   phi(prey_captured,xp_level) |
+==================+====================+===============================+
| prey_captured(1) | xp_level(advanced) |                        0.2396 |
+------------------+--------------------+-------------------------------+
| prey_captured(1) | xp_level(interm)   |                        0.3642 |
+------------------+--------------------+-------------------------------+
| prey_captured(1) | xp_level(novice)   |                        0.1744 |
+------------------+--------------------+-------------------------------+
| prey_captured(0) | xp_level(advanced) |                        0.0774 |
+------------------+--------------------+-------------------------------+
| prey_captured(0) | xp_level(interm)   |                        0.1002 |
+------------------+--------------------+-------------------------------+
| prey_captured(0) | xp_level(novice) 

Ultimately, it seems that the probability of capturing prey remains mostly the same regardless of whether the time lag is very long or short. Thus quickly acquiring experience does not seem to be important at early stages and players may still perform similarly regardless of longer intervals between game sessions (e.g. forgetting crucial info, muscle memory, etc.).

**Keep this for question 3 when I do chase rate and xp level**

As expected, chances of capturing prey are lower while chances of not capturing prey increase at high chase rates. While this may seem counterintuitive, it indicates that while such predators may encounter and chase prey often, they do not optimize their chases and probably fail most of them. This may also indicate that such predators get mobbed by the prey a lot.

## **4. Question 2 - Past experience and memory**

In [23]:
q4 = infer.query(
    variables = ["pred_speed_bin", "iqrange_prey_speed_bin"],
    joint = True,
    n_samples = 5000,
    seed = 123,
    show_progress = False
)

In [24]:
print(q4)

+-------------------------+----------------------------+----------------------------------------------+
| pred_speed_bin          | iqrange_prey_speed_bin     |   phi(pred_speed_bin,iqrange_prey_speed_bin) |
+=========================+============================+==============================================+
| pred_speed_bin(Average) | iqrange_prey_speed_bin(Q5) |                                       0.1302 |
+-------------------------+----------------------------+----------------------------------------------+
| pred_speed_bin(Average) | iqrange_prey_speed_bin(Q4) |                                       0.1436 |
+-------------------------+----------------------------+----------------------------------------------+
| pred_speed_bin(Average) | iqrange_prey_speed_bin(Q3) |                                       0.1568 |
+-------------------------+----------------------------+----------------------------------------------+
| pred_speed_bin(Average) | iqrange_prey_speed_bin(Q1) |        

The probability of adopting an intercepting foraging mode is much higher than other modes regardless of the range of prey encountered.

Let's see if the foraging mode changes with past success.

In [25]:
q5 = infer.query(
    variables = ["pred_speed_bin", "prior_prey_captured"],
    joint = True,
    n_samples = 5000,
    seed = 123,
    show_progress = False
)

In [26]:
print(q5)

+-------------------------+--------------------------+-------------------------------------------+
| pred_speed_bin          | prior_prey_captured      |   phi(pred_speed_bin,prior_prey_captured) |
+=========================+==========================+===========================================+
| pred_speed_bin(Average) | prior_prey_captured(1.0) |                                    0.5570 |
+-------------------------+--------------------------+-------------------------------------------+
| pred_speed_bin(Average) | prior_prey_captured(0.0) |                                    0.1500 |
+-------------------------+--------------------------+-------------------------------------------+
| pred_speed_bin(Faster)  | prior_prey_captured(1.0) |                                    0.0750 |
+-------------------------+--------------------------+-------------------------------------------+
| pred_speed_bin(Faster)  | prior_prey_captured(0.0) |                                    0.0222 |
+---------

Chances of adopting an intercepting strategy are much higher when predators previously captured prey. This is because at the population level, it is the optimal strategy. Again, when no captures occured, predators are also more likely to adopt the intercepting strategy, probably because it is the best at managing any type of prey overall.